In [ ]:
import rasterio
import numpy as np
import xarray as xr
import hvplot.xarray
import holoviews as hv
import geoviews as gv
import rioxarray as rx
import os
import panel as pn
from holoviews import opts
hv.extension('bokeh', width=90)
pn.extension()
import xrspatial as xrs
from holoviews.streams import SingleTap
from decimal import Decimal
from math import isnan

In [ ]:
places={}
places['St.Thomas']="https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n19w065/USGS_1_n19w065.tif"
places['St.Croix']="https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n18w065/USGS_1_n18w065.tif"
places['Cornell']='https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n42w076/USGS_1_n42w076.tif'

In [ ]:
maps = ['inferno','blues','kb','kgy','viridis']

# Creating variables which will store shared value

In [ ]:
data = data=xr.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n19w065/USGS_1_n19w065.tif")
sloped =0
maxheight = 0
minheight = 0
maxslope = 0
minslope = 0
maxaspect = 0
minaspect = 0
mincurve = 0
maxcurve = 0
test = ''
link = ''

# Making useful helper functions

In [ ]:
def createlink(x,y):
    global link
    link = 'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/'
    north = "{:02d}".format(round(abs(y)))
    west =  "{:03d}".format(round(abs(x)))
    coordinates = 'n'+north+'w'+west
    finallink = link+coordinates+'/USGS_1_'+coordinates+'.tif'
    link = finallink
    return (finallink,coordinates) 

In [ ]:
def getmaxmin(dset):
    return (round(float(dset.min())),round(float(dset.max())))

In [ ]:
def getpercentage(orig,new):
    origcount = int((~np.isnan(orig.values)).sum())
    newcount = int((~np.isnan(new.values)).sum())
    return round((newcount/origcount)*100)

# Making widgets and Text Displays

In [ ]:
#locationwidget = pn.widgets.Select(name='Select Location', options=places)
cmapwidget = pn.widgets.Select(name='Select Color Map', options=maps)
heightquantileslider = pn.widgets.IntSlider(name='Height Quantile Slider', start=0, end=32, value=7)
heightrangeslider = pn.widgets.RangeSlider(step=1,start = minheight, end = maxheight, value=(minheight,maxheight))
sloperangeslider = pn.widgets.RangeSlider(end=maxslope,step=1,value=(minslope,maxslope))
aspectrangeslider = pn.widgets.RangeSlider(start = minaspect, end=maxaspect,step=1,value=(minaspect,maxaspect))
curveaturerangeslider = pn.widgets.RangeSlider(start = mincurve, end=maxcurve,step=1,value=(mincurve,maxcurve))

In [ ]:
heightp = pn.pane.Markdown()
heightpercentpanel = pn.Column(heightp,css_classes=['panel-widget-box'],margin=25)

# Creating the displays for each tab

In [ ]:
tiles = gv.tile_sources.EsriImagery().opts(width=600, height=550).redim.range(Latitude=(-50,50), Longitude=(-50,50))
#tiles = gv.tile_sources.EsriImagery().opts(width=600, height=550).opts(xlim=(-50, 50), ylim=(-50, 50))

tap_stream = SingleTap(source=tiles, x=65, y=19)

@pn.depends(tap_stream.param.x, tap_stream.param.y)
def lat_lon(x,y):
    return f"### Lat: {y:.4f} Lon: {x:.4f}"

def interactive_click(x, y):
    return gv.Points([(x,y)])
click = hv.DynamicMap(interactive_click, streams=[tap_stream])

In [ ]:
@pn.depends(colormap = cmapwidget,x=tap_stream.param.x, y=tap_stream.param.y)
def options(colormap,x,y):
    global data
    global test
    try:
        place = createlink(x,y)[0]
        data = rx.open_rasterio(place, masked=True).squeeze().astype('float64')
        data.attrs = xr.open_rasterio(place).attrs
        data = data.where(data>0)
        test = 'All good!'
    except:
        test = 'There was an error'

In [ ]:
@pn.depends(x=tap_stream.param.x, y=tap_stream.param.y)
def shaded(x,y):
    global data
    shade = xrs.hillshade(data)
    return shade.hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = cmapwidget, alpha = 20)

In [ ]:
@pn.depends(x=tap_stream.param.x, y=tap_stream.param.y, level = heightquantileslider)
def quantile(x,y, level):
    global data
    quant = xrs.quantile(data,k=level,ignore_vals=(0,))
    return quant.hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = cmapwidget, alpha = 20)

In [ ]:
@pn.depends(x=tap_stream.param.x, y=tap_stream.param.y, border = heightrangeslider)
def heightrange(x,y, border):
    global maxheight
    global minheight
    percentheight = 0
    global data
    (minheight,maxheight) = getmaxmin(data)
    heightrangeslider.start = minheight
    heightrangeslider.end = maxheight
    manipulate = data.where(np.logical_and(data>border[0],data<border[1]))
    heightp.object = '# Your selected range is '+ str(getpercentage(data,manipulate))+'% of this region'
    shade = xrs.hillshade(manipulate)    
    return shade.hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = cmapwidget)

In [ ]:
@pn.depends(x=tap_stream.param.x, y=tap_stream.param.y, border = sloperangeslider)
def slope(x,y, border):
    global maxslope
    global minslope
    global data
    sloped = xrs.slope(data)
    (minslope,maxslope) = getmaxmin(sloped)
    sloperangeslider.start = minslope
    sloperangeslider.end = maxslope
    manipulate = sloped.where(np.logical_and(sloped>border[0],sloped<border[1]))
    return manipulate.hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = cmapwidget)

In [ ]:
@pn.depends(x=tap_stream.param.x, y=tap_stream.param.y, border = aspectrangeslider)
def aspect(x,y, border):
    global maxaspect
    global minasepct
    global data
    aspects = xrs.aspect(data)
    (minaspect,maxaspect) = getmaxmin(data)
    aspectrangeslider.start = minaspect
    aspectrangeslider.end = maxaspect
    manipulate = aspects.where(np.logical_and(aspects>border[0],aspects<border[1]))
    return manipulate.hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = cmapwidget)

In [ ]:
@pn.depends(x=tap_stream.param.x, y=tap_stream.param.y, border = curveaturerangeslider)
def curvature(x,y, border):
    global maxcurve
    global mincurve
    global data
    curvature = xrs.aspect(data)
    (mincurve,maxcurve) = getmaxmin(data)
    curveaturerangeslider.start = mincurve
    curveaturerangeslider.end = maxcurve
    manipulate = curvature.where(np.logical_and(curvature>border[0],curvature<border[1]))
    return manipulate.hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = cmapwidget)

# Creating the canvas for each tab

In [ ]:
selector = pn.Column(pn.panel(lat_lon, width=500), (tiles * click))
option = pn.Column(options,selector,cmapwidget)
quantileview = pn.Row(quantile, heightquantileslider)
heightrangeview = pn.Row(heightrange, pn.Column(heightrangeslider,heightpercentpanel))
slopeview = pn.Row(slope, sloperangeslider)
aspectview = pn.Row(aspect, aspectrangeslider)
curvatureview = pn.Row(curvature, curveaturerangeslider)

In [ ]:
dashboard = pn.Tabs(('Options',option), 
                    ("Shaded View", shaded),
                    ('Height Range', heightrangeview),
                    ('Height Quantile', quantileview),
                    ('Aspect View', aspectview),
                    ('Slope View', slopeview),
                    ('Curvature View', curvatureview),
                    dynamic = True
                   )

In [ ]:
dashboard.servable()

In [ ]:
link

In [ ]:
test